In [25]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.


In [11]:
# Communications according to:
# https://arduino.stackexchange.com/questions/23684/stpm10-using-arduino-spi-communication/23742

# spi outline here: https://forum.micropython.org/viewtopic.php?f=16&t=1705&sid=a941eec9bce447f4046c2bd164953481&start=10


In [26]:
import machine

SPISCS = machine.Pin(27, machine.Pin.OUT)
SPISDA = machine.Pin(12)
SPISCL = machine.Pin(14, machine.Pin.OUT)
SPISYN = machine.Pin(26, machine.Pin.OUT)
SPISCS.value(1)
SPISYN.value(1)
spi = machine.SPI(miso=SPISDA, mosi=machine.Pin(13, machine.Pin.OUT), sck=SPISCL, 
                  baudrate=32000, firstbit=machine.SPI.MSB)
spi.init()


In [13]:
import time
help(time)

object <module 'utime'> is of type module
  __name__ -- utime
  localtime -- <function>
  mktime -- <function>
  time -- <function>
  sleep -- <function>
  sleep_ms -- <function>
  sleep_us -- <function>
  ticks_ms -- <function>
  ticks_us -- <function>
  ticks_cpu -- <function>
  ticks_add -- <function>
  ticks_diff -- <function>


In [30]:
import time
def write_reg(addr, val):
    val = (val << 7) | (addr << 1);  # 6-bit addresses!
    spi.deinit()
    SPISDA.init(machine.Pin.OUT)
    SPISYN.value(1)
    time.sleep_us(10)
    SPISCS.value(0)
    SPISYN.value(0)
    for i in range(8):
        time.sleep_us(10);
        SPISCL.value(0)
        SPISDA.value(1 if (val & 0x80) else 0);
        time.sleep_us(10);
        SPISCL.value(1)
        val <<= 1;

    SPISYN.value(1)
    SPISCS.value(1)
    SPISDA.init(machine.Pin.IN)
    spi.init()

def read_regs(n):
    SPISYN.value(0)
    SPISCS.value(0)
    SPISYN.value(1)
    res = [ ]
    for i in range(n):
        res.append(spi.read(4))
    SPISCS.value(1)
    return res

In [31]:
SPISDA.init(SPISDA.OUT)
print(SPISDA.value())


0


In [19]:
print(spi.read(4))


b'\x00\x00\x00\x00'


In [33]:

write_reg(47, 1) #;  // set config bit 47
x = read_regs(8) #;  // read all reg data into array
print(x)


[b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00']
